# Welcome to the Day 2 Lab!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get started --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## First - let's talk about the Chat Completions API

1. The simplest way to call an LLM
2. It's called Chat Completions because it's saying: "here is a conversation, please predict what should come next"
3. The Chat Completions API was invented by OpenAI, but it's so popular that everybody uses it!

### We will start by calling OpenAI again - but don't worry non-OpenAI people, your time is coming!


In [2]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENROUTER_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-or-"):
    print("An API key was found, but it doesn't start sk-or- (OpenRouter format); please check you're using OPENROUTER_API_KEY - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


## Do you know what an Endpoint is?

If not, please review the Technical Foundations guide in the guides folder

And, here is an endpoint that might interest you...

In [3]:
import requests

headers = {"Authorization": f"Bearer {api_key}", "Content-Type": "application/json"}

payload = {
    "model": "gpt-5-nano",
    "messages": [
        {"role": "user", "content": "Tell me a fun fact"}]
}

payload

{'model': 'gpt-5-nano',
 'messages': [{'role': 'user', 'content': 'Tell me a fun fact'}]}

In [5]:
# Use OpenRouter endpoint (your OPENROUTER_API_KEY only works here, not at api.openai.com)
response = requests.post(
    "https://openrouter.ai/api/v1/chat/completions",
    headers=headers,
    json=payload
)

response.json()

{'id': 'gen-1771996100-sXVKzaR8UwIOADPMoaLR',
 'provider': 'OpenAI',
 'model': 'openai/gpt-5-nano',
 'object': 'chat.completion',
 'created': 1771996100,
 'choices': [{'logprobs': None,
   'finish_reason': 'stop',
   'native_finish_reason': 'completed',
   'index': 0,
   'message': {'role': 'assistant',
    'content': 'Fun fact: Honey never spoils. Archaeologists have found pots of honey in ancient Egyptian tombs that are thousands of years old and still edible. Want another?',
    'refusal': None,
    'reasoning': '**Providing a fun fact**\n\nThe user wants a fun fact, so I need to pick something succinct and interesting. I’m considering options like "Honey never spoils," which is a classic. It\'s true that archaeologists have found 3,000-year-old honey still edible due to its low water activity and acidity. I\'m also thinking about others, like "A group of flamingos is a flamboyance," or "Wombats produce cube-shaped poo." I\'ll stick to one and share more if the user is curious!**Cra

In [6]:
response.json()["choices"][0]["message"]["content"]

'Fun fact: Honey never spoils. Archaeologists have found pots of honey in ancient Egyptian tombs that are thousands of years old and still edible. Want another?'

# What is the openai package?

It's known as a Python Client Library.

It's nothing more than a wrapper around making this exact call to the http endpoint.

It just allows you to work with nice Python code instead of messing around with janky json objects.

But that's it. It's open-source and lightweight. Some people think it contains OpenAI model code - it doesn't!


In [7]:
# Create OpenAI-compatible client pointing at OpenRouter (uses OPENROUTER_API_KEY)

from openai import OpenAI
openai = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=api_key)

response = openai.chat.completions.create(model="gpt-5-nano", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content



'Fun fact: Honey never spoils. Archaeologists have found edible honey in ancient Egyptian tombs that’s thousands of years old. Want another one in a specific category?'

## And then this great thing happened:

OpenAI's Chat Completions API was so popular, that the other model providers created endpoints that are identical.

They are known as the "OpenAI Compatible Endpoints".

For example, google made one here: https://generativelanguage.googleapis.com/v1beta/openai/

And OpenAI decided to be kind: they said, hey, you can just use the same client library that we made for GPT. We'll allow you to specify a different endpoint URL and a different key, to use another provider.

So you can use:

```python
gemini = OpenAI(base_url="https://generativelanguage.googleapis.com/v1beta/openai/", api_key="AIz....")
gemini.chat.completions.create(...)
```

And to be clear - even though OpenAI is in the code, we're only using this lightweight python client library to call the endpoint - there's no OpenAI model involved here.

If you're confused, please review Guide 9 in the Guides folder!

And now let's try it!

## THIS IS OPTIONAL - but if you wish to try out Google Gemini, please visit:

https://aistudio.google.com/

And set up your API key at

https://aistudio.google.com/api-keys

And then add your key to the `.env` file, being sure to Save the .env file after you change it:

`GOOGLE_API_KEY=AIz...`


In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

load_dotenv(override=True)

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")



In [ ]:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

response = gemini.chat.completions.create(model="gemini-2.5-flash-lite", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

## And Ollama also gives an OpenAI compatible endpoint

...and it's on your local machine!

If the next cell doesn't print "Ollama is running" then please open a terminal and run `ollama serve`

In [ ]:
requests.get("http://localhost:11434").content

### Download llama3.2 from meta

Change this to llama3.2:1b if your computer is smaller.

Don't use llama3.3 or llama4! They are too big for your computer..

In [ ]:
!ollama pull llama3.2

In [ ]:
OLLAMA_BASE_URL = "http://localhost:11434/v1"

ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key='ollama')


In [ ]:
# Get a fun fact

response = ollama.chat.completions.create(model="llama3.2", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

In [ ]:
# Now let's try deepseek-r1:1.5b - this is DeepSeek "distilled" into Qwen from Alibaba Cloud

!ollama pull deepseek-r1:1.5b

In [ ]:
response = ollama.chat.completions.create(model="deepseek-r1:1.5b", messages=[{"role": "user", "content": "Tell me a fun fact"}])

response.choices[0].message.content

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

## Solution: Webpage summarizer with Ollama

Below we use the **week1 scraper** to fetch a page and **Ollama** (the `ollama` client from above) to summarize it. Run from repo root or from this folder; ensure `ollama serve` is running and you have run `ollama pull llama3.2` (or use `llama3.2:3b-instruct-q4_0` / `llama3.2:1b` — see `ollama list`).

In [10]:
# Path so we can import week1 scraper (run from repo root or community-contributions/asket)
import os, sys
for _path in ("week1", os.path.join(os.getcwd(), "..", "..", "week1")):
    _p = os.path.abspath(_path)
    if _p not in sys.path and os.path.isdir(_p):
        sys.path.insert(0, _p)
        break
from scraper import fetch_website_contents
from openai import OpenAI

# Ollama client (so this cell runs even if you didn't run the earlier Ollama cells)
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

# Try these model tags in order; first one that Ollama has will be used
OLLAMA_MODELS_TO_TRY = ["llama3.2:3b-instruct-q4_0", "llama3.2:1b-instruct-q4_0", "llama3.2:1b", "llama3.2"]

def summarize_url(url):
    from openai import NotFoundError
    text = fetch_website_contents(url)
    messages = [
        {"role": "system", "content": "Summarize the following webpage content in a few clear bullet points. No code blocks."},
        {"role": "user", "content": text}
    ]
    for model in OLLAMA_MODELS_TO_TRY:
        try:
            r = ollama.chat.completions.create(model=model, messages=messages)
            return r.choices[0].message.content
        except NotFoundError:
            continue
    raise RuntimeError("No Ollama model found. Run: ollama pull llama3.2")

# Example: summarize a page (change URL as you like)
url = "https://github.com/frank-asket"
print(summarize_url(url))

Here are three clear bullet points summarizing the content:

* GitHub offers a range of tools and features to help developers create, manage, and secure code, including AI-powered assistances like Copilot, automated workflows, and issue tracking.
* The platform caters to different users and use cases, including enterprises, small and medium teams, startups, nonprofits, healthcare, financial services, manufacturing, and government.
* GitHub also provides resources for learning and support, such as documentation, customer support, community forums, and training programs like the Security Lab and Accelerator.
